In [1]:
from sqlalchemy import create_engine, event
import urllib.parse

import pandas as pd
import datetime as dt
import numpy as np
import time

# Input & Output

In [2]:
# SQL database
server = 'CSKMA0400\RDB_Data'
db = 'JLDJobPath'
odbc_connection_string = 'DRIVER={SQL Server Native Client 11.0};SERVER='+server+';DATABASE='+db+';Trusted_Connection=yes'

#Input Table
input_sql_table = "linkedclaims_casuals_2018m04_v2_flat_20160101_with_income_36Vars__7BGM_full_clusters_jp_summary_with_outcomes_edu_selected_jld"

output_csv_file = "//cskma0294/F/Evaluations/JobPath/Python/Data/OutputAnalysis/"+input_sql_table+".csv"
out_cols_drop = []
exportIndex = False

# Procedure

In [3]:
def read_data_from_sql(sql_table):
    # Connect to SQL
    params = urllib.parse.quote_plus(odbc_connection_string)
    engine = create_engine('mssql+pyodbc:///?odbc_connect=%s' % params)
    conn = engine.connect().connection

    @event.listens_for(engine, 'before_cursor_execute')
    def receive_before_cursor_execute(conn, cursor, statement, params, context, executemany):
        if executemany:
            cursor.fast_executemany = True

    df = pd.read_sql_table(sql_table, engine)
    conn.close()
    return df


data = read_data_from_sql(input_sql_table)
if len (out_cols_drop) > 0:
    for elem in out_cols_drop:
        data.drop(columns=elem)
data.to_csv(output_csv_file, index=exportIndex)

print ('ALL DONE!')

ALL DONE!
